# Weekly Report
- Team members: 21700442 오경택, 21700741 최종현 22000624 장승민
- Meeting Date: 230911
- Project Name: 공용자전거 대여 데이터를 기반으로한 서울시 신설 최적의 자전거도로 입지 제안
- Project Goal: GCN 모델(공간,시간 데이터 사용)을 활용한 대여소별 서울시 공용자전거 대여량 예측

### Weekly plan (Fill this in)
| Week    | Schedule        | Note                      |
|---------|-----------------|---------------------------|
| Week 1  | 간이 GCN 모델 코드 동작, GPU 개선, LSTM 코드 개선  | 종현: 간이 GCN 모델 코드 동작 완료, 사전 논문 전처리 코드 이해 및 적용 완료 <br> 승민: LSTM 코드 구현 완료 <br> 경택: conda를 docker 변환, ARIMA 모델 코드 구현 |
| Week 2  | 간이 GCN 모델 구현 완료, LSTM 코드 구현 완료, GCN을 제외한 다른 비교 모델 코드 구현 시작 |종현: GCN 라플라시안 행렬 적용, 날씨, 거치소 별 공간적 변수 적용 <br> 승민: LSTM 코드 구현 <br> 경택: 아리마 코드 구현 |
| Week 3  | GCN 모델 구현 완료, GCN을 제외한 다른 비교 모델 코드 구현 진행 | 종현: GCN sliding window 적용, GCN 주별 데이터 포함 <br> 승민: LSTM 코드 구현, 자전거도로 데이터 처리 <br> 경택: 아리마, xG boost 코드 구현      |
| Week 4  | GCN을 제외한 다른 비교 모델 코드 구현 완료 | 종현: GCN sliding window 적용, GCN 하이퍼파라미터 최적화 <br> 승민: 날씨데이터 4분할, 자전거도로 데이터 처리 <br> 경택: 아리마, xG boost 하이퍼파라미터 최적화 | 
| Week 5  | 논문 작성, 모델 점검 및 수정, 사후 점검 | 종현: 1차 GCN 하이퍼파라미터 최적화 <br> 승민: 시계열 데이터분리, LSTM 모델 구현 <br> 경택: Sliding window 구현                          |
| Week 6  | 논문 작성, 모델 점검 및 수정, 사후 점검 |                           |
| Week 7  | 논문 작성, 모델 점검 및 수정, 사후 점검      |                           |
| Week 8  |                  |                           |
| Week 9  |                  |                           |
| Week 10 |                 |                           |
| Week 11 |                 |                           |
| Week 12 |                 |                           |
| Week 13 |                 |                           |
| Week 14 |                 |                           |
| Week 15 |                 |                           |
| Week 16 |                 |                           |


### Summary of previous meeting
- Summary of previous meeting
- Previous discussion:
  - 거치소 인근 자전거 도로 개수 포함 문제를 자전거 도로의 수선의 발을 그은 수직선과의 거리를 계산해서 해결
  - GCN 1차 하이퍼파라미터 최적화 완료

- Comments from:
  - GCN 하이퍼파라미터 조정할 때, 단일 하이퍼파라미터만으로 조정하는 것보다 여러개의 하이퍼파라미터의 조합으로 조정을 해보아라.
  - 공간행렬, 날씨 feature 중요도를 고려해서 반영해보는 것도 좋을 것이다.

- Previous plan:
  - 하이퍼파라미터 여러 개의 조합으로 최적화해보기
  - LSTM, xGboost 모델 보완

### Summary of working progress  

#### process 1: 자전거도로 데이터
- 지난 데이터: (A) type, (B) type 자전거도로 구분하여 제작
- (A) type: 도심 속 or 도로변에 있는 자전거도로
- (B) type: 하천(한강 등) 주변에 있는 자전거도로
- 서로 분리해서 진행 결정 -> 사람들이 이용하는 수요와 목적이 다를 것

#### process 2: LSTM, XG-Boost
- 시계열 데이터 분리 문제
  - case 1: X(날씨), y(대여량)으로 설정한 후 train, test 데이터 분리
    - 데이터셋
      - train_X: 3일치의 날씨 데이터: (361,3,9)
      - train_y: 3일치의 자전거대여량 데이터: (361,3)
      - test_X: 당일날의 날씨 데이터: (361,9)
      - test_y: 당일날의 자전거대여량 데이터: (361,)
    - 발생하는 문제: train의 X와 test의 X의 time_steps이 다름
      - train_X를 3일로 잡고 진행했기 때문에, test_X도 3일의 time_steps를 가져야함
      - XGBoost의 경우 후보 sol: input으로 3D가 들어가지 못하므로 train_X의 shape을 2D로 변경: (361,3,9) -> (361,27) 방법 문의

  - case 2: X(3일치의 대여량+당일의 날씨), y(당일의 대여량)으로 설정
    - training 과정을 정상적으로 진행됨
    - 발생하는 문제: test set 분리
      - training에서 진행됐던 데이터를 사용하여 test에 적용할 수 없음
      - 모든 날짜에 대한 training이 불가능해짐

  - 위 2가지 case로 시계열 데이터를 분리하여 진행하였지만, 문제점을 가지고 있어 어떻게 해결하는 것이 좋을지 고민중

#### process 3: GPU
- GPU가 안되고 있었던 이유: 컨네이너를 만들지 않고 코드를 돌리고 있어서이다. 승민이가 만든 계정은 gpu가 할당된 컨네이너를 만들어서 승민이 서버로 코드를 돌려볼 예정이다.

### Pending assignments or requests
- pending assingments: 
  - GCN 하이퍼파라미터 최적화 이슈
    - 현재 6개 거치소의 5월31일 하루를 예측하는 것에 대해서 hyper parameter 최적화를 하고 있다. 하지만 2600개의 거치소에서 6개의 거치소만, 그리고 365일 중에 1일만을 이용해서 하이퍼파라미터 최적화 하는 것이 좋은 방법이 아닌 것 같다.

    - 그래서 6개의 거치소와 1일을 에측할 때 크게 영향을 주는 하아퍼파라미터는 일정하게 가져가고, 최적화의 차이가 크게 나지 않는 하이퍼파라미터의 한해서 전체로 돌려보는 것으로 생각하고 있다. 어떻게 생각하시는지 궁금하다.
- requests: 
